# Social Network Analysis - Exercise Sheet 5b)


### Multi-Relational Networks (MRGraphs)

In this task, you will work with MRGraphs, compositions of relations and their underlying algebraic structure. You will test and demonstrate your implementations on real world data. For the different tasks below, you can or several data sets, if this makes it easier to demonstrate your implementations.

#### Guidelines
* Submit your code zipped via [moodle](https://moodle.uni-kassel.de/course/view.php?id=11038) until 26.01.2023 23:55 MEZ
* Use the [NetworkX](https://networkx.github.io/documentation/stable/) library for your graphs.

##### Exercise:
1. Think of how MRGraph can be implemented. You can use networkx or implement your own.
2. Think of some example real world MRGraph with three different relations and implement it (load it into the jupyter notebook).
3. Provide or implement some visualization for your MultiGraph.
4. Implement structural equivalence for MRGraphs. Apply this to your graph. What does this show you?
5. Implement the graph composition (compound relation) of two relations. Your implementation should be able to compose  arbitrarily many further relations.
6. Apply the composition to some example in your graph. What are your findings?
6. Implement the Construction Algorithm for MRG induced Semigroups and apply it to your data. Why is this useful?